In [10]:
#locale fix sourced from Stephen Ng'etich on SO
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import pandas as pd
#!pip install fastparquet
import fastparquet
import spacy, en_core_web_sm
#!pip install langdetect
from langdetect import detect
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.parsing import preprocess_string, strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#import version licensed from Deconstructing Game Design
game_df = pd.read_parquet('https://github.com/canunj/deconstructing_games/blob/main/ranked_df.parquet.gzip?raw=true', engine='fastparquet')

In [ ]:
print(game_df.columns[0:33])
game_df.sample(5)

Index(['bgg_id', 'name', 'rank', 'year', 'alt_name', 'artist', 'avg_rating',
       'bayes_rating', 'category', 'complexity', 'description', 'designer',
       'family', 'game_type', 'language_dependency', 'max_players',
       'max_players_best', 'max_players_rec', 'max_time', 'mechanic',
       'min_age', 'min_age_rec', 'min_players', 'min_players_best',
       'min_players_rec', 'min_time', 'num_votes', 'publisher',
       'stddev_rating', 'url', 'video_url', 'expansion', 'cooperative'],
      dtype='object')


,bgg_id,name,rank,year,alt_name,artist,avg_rating,bayes_rating,category,complexity,...,artist_爪尾 (Tsumeo),artist_猫背,artist_田中寛崇 (Tanaka Hirotaka),artist_純志 (Junsi),artist_舟岡 (Hunaoka),artist_苍月白龙（Cangyue Bailong）,artist_菅原 美沙穂 (Misaho Sugawara),artist_都 和 (Nagi Miyako),artist_長谷川 登鯉 (Tori Hasegawa),artist_ＡＲＩ
49602,112844,Swintus 2.0,17994,2011,"[Swintus 2.0, Свинтус 2.0]",[Sergey Dulin],5.73038,5.50217,"['Animals', 'Card Game', 'Number', 'Party Game']",1.2500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80519,247107,Catchup & Mousetard: Fast Food Battle!,15259,2019,"[Catchup & Mousetard: Fast Food Battle!, Fast ...",[Sergio Herrera Jiménez],6.94540,5.51721,"['Animals', 'Card Game', 'Humor', 'Party Game']",1.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35471,41149,Linkage,11002,2009,[Linkage],[Néstor Romeral Andrés],6.56321,5.54520,['Abstract Strategy'],2.1111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95031,315138,Autumn Harvest: A Tea Dragon Society Game,8205,2020,[Autumn Harvest: A Tea Dragon Society Game],"[Neill"", ]",7.14426,5.58487,"['Comic Book / Strip', 'Fantasy']",1.2500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75124,219661,Crop Rotation,14888,2016,[Crop Rotation],None,6.59474,5.51923,"['Abstract Strategy', 'Farming', 'Print & Play']",1.6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(game_df)

22192

##Text Processing and Language Detection

In [8]:
#customized text processor by N.Canu & K.Chen licensed from Deconstructing Game Design
def doc_text_preprocessing(ser):
  nlp=spacy.load("en_core_web_sm", exclude=['parser','ner','textcat'])

  """text processing steps"""
  import re
  stop_words=set(stopwords.words('english'))
  stop_words.update(['game','player','players','games', 'also', 
                     'description','publisher'])
  
  single_letter_replace=lambda c: re.sub("\s+\w{1}\s+|\n|-|—",'',c)
  to_lower_func=lambda c: c.lower()

  lemma_text=[preprocess_string(
      ' '.join([token.lemma_ for token in desc]
          ),[remove_stopwords,strip_numeric,strip_punctuation,strip_tags,
             strip_multiple_whitespaces,single_letter_replace,to_lower_func]
             ) for desc in ser.apply(lambda x: nlp(x))]

  tokenize_text=[[word for word in string if word not in stop_words] for string in lemma_text]

  return tokenize_text


In [11]:
nlp=en_core_web_sm.load()
game_df['description']=game_df['description'].fillna('no words')
game_df['cleaned_descriptions']=doc_text_preprocessing(game_df['description'])

In [14]:
detected_lang = []
for word in game_df.cleaned_descriptions:
  word=', '.join(word)
  detected_lang.append(detect(word))

In [17]:
game_df['lang'] = detected_lang

In [20]:
game_df[game_df['lang']=='en'].to_parquet('Games_w_Eng_Desc.parquet.gzip',compression='gzip')